In [24]:
# import all necessary dependencies
import pandas as pd
import pathlib
from pathlib import Path
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.orm import Session
import matplotlib.pyplot as plt
from sqlalchemy.types import String, Float, Integer
import psycopg2
import sklearn as skl
import tensorflow as tf
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler

In [25]:
# read in the four raw data files
season_2015_df = pd.read_excel('wr_season_2015.xlsx')
season_2016_df = pd.read_excel('wr_season_2016.xlsx')
season_2017_df = pd.read_excel('wr_season_2017.xlsx')
season_2018_df = pd.read_excel('wr_season_2018.xlsx')
season_2019_df = pd.read_excel('wr_season_2019.xlsx')
season_2020_df = pd.read_excel('wr_season_2020.xlsx')
season_2021_df = pd.read_excel('wr_season_2021.xlsx')

## Data Cleaning

In [26]:
# preview new dataframe
season_2015_df.head()

,Rk,Name,Team,Pos,GMS,TGTS,REC,PCT,YDS,TD,...,Y/T,Y/R,ATT,YDS.1,AVG,TD.1,FUM,LST,FPTS/G,FPTS
0,1,Antonio Brown,PIT,WR,16,193,136,70.5,1834,10,...,9.5,13.5,3,28,9.3,0,3,2,15.4,246.2
1,2,Julio Jones,ATL,WR,16,203,136,67.0,1871,8,...,9.2,13.8,0,0,0.0,2,3,1,15.3,245.1
2,3,Brandon Marshall,NYJ,WR,16,173,109,63.0,1502,14,...,8.7,13.8,0,0,0.0,0,3,2,14.4,230.2
3,4,Allen Robinson II,JAX,WR,16,151,80,53.0,1400,14,...,9.3,17.5,0,0,0.0,0,0,0,14.0,224.0
4,5,Odell Beckham Jr.,NYG,WR,15,158,96,60.8,1450,13,...,9.2,15.1,1,3,3.0,0,2,0,14.9,223.3


In [27]:
# drop rank column from all dataframes since we don't need this value for the model
season_2015_df = season_2015_df.drop(columns=['Rk'])
season_2016_df = season_2016_df.drop(columns=['Rk'])
season_2017_df = season_2017_df.drop(columns=['Rank'])
season_2018_df = season_2018_df.drop(columns=['Rank'])
season_2019_df = season_2019_df.drop(columns=['Rank'])
season_2020_df = season_2020_df.drop(columns=['Rank'])
season_2021_df = season_2021_df.drop(columns=['Rank'])

In [28]:
# rename columns to be clear terminology
season_2015_df = season_2015_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [29]:
# rename columns to be clear terminology
season_2016_df = season_2016_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [30]:
# rename columns to be clear terminology
season_2017_df = season_2017_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [31]:
# rename columns to be clear terminology
season_2018_df = season_2018_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [32]:
# rename columns to be clear terminology
season_2019_df = season_2019_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [33]:
# rename columns to be clear terminology
season_2020_df = season_2020_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'TGTS': 'Targets', 'REC': 'Receptions', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [34]:
# rename columns to be clear terminology
season_2021_df = season_2021_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'TGTS': 'Targets', 'REC': 'Receptions', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [35]:
# check to make sure columns have been renamed
season_2015_df

,Name,Team,Position,Games,Targets,Receptions,Percentage,Yards,Touchdowns,Long,Yards_per_target,Yards_per_reception,Attempts,Rushing_yards,Average_rushing_yards,Rushing_touchdown,Fumbles,Lost_yards,Fantasy_points_per_game,Fantasy_points
0,Antonio Brown,PIT,WR,16,193,136,70.5,1834,10,59,9.5,13.5,3,28,9.3,0,3,2,15.4,246.2
1,Julio Jones,ATL,WR,16,203,136,67.0,1871,8,70,9.2,13.8,0,0,0.0,2,3,1,15.3,245.1
2,Brandon Marshall,NYJ,WR,16,173,109,63.0,1502,14,69,8.7,13.8,0,0,0.0,0,3,2,14.4,230.2
3,Allen Robinson II,JAX,WR,16,151,80,53.0,1400,14,90,9.3,17.5,0,0,0.0,0,0,0,14.0,224.0
4,Odell Beckham Jr.,NYG,WR,15,158,96,60.8,1450,13,87,9.2,15.1,1,3,3.0,0,2,0,14.9,223.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,Je'Ron Hamm,SF,WR,1,0,0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,0,0,0.0,0.0
212,Chris Harper,NE,WR,5,3,1,33.3,6,0,6,2.0,6.0,0,0,0.0,0,1,1,-0.3,-1.4
213,Denarius Moore,BUF,WR,6,0,0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,2,1,-0.3,-2.0
214,Frankie Hammond,KC,WR,9,0,0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,2,1,-0.2,-2.0


In [36]:
# add column to each dataframe that contains the year of play
season_2015_df['Year'] = '2015'
season_2016_df['Year'] = '2016'
season_2017_df['Year'] = '2017'
season_2018_df['Year'] = '2018'
season_2019_df['Year'] = '2019'
season_2020_df['Year'] = '2020'
season_2021_df['Year'] = '2021'

In [37]:
# all_dfs = [season_2015_df, season_2016_df, season_2017_df, season_2018_df, season_2019_df, season_2020_df, season_2021_df]
# results = pd.concat(all_dfs)
all_dfs = [season_2018_df, season_2019_df, season_2020_df, season_2021_df]
results = pd.concat(all_dfs)

In [38]:
# write the clean dataframes to csv for future use
season_2015_df.to_csv('wr_season_2015_clean.csv', index=False)

In [39]:
# write the clean dataframes to csv for future use
season_2016_df.to_csv('wr_season_2016_clean.csv', index=False)

In [40]:
# write the clean dataframes to csv for future use
season_2017_df.to_csv('wr_season_2017_clean.csv', index=False)

In [41]:
# write the clean dataframes to csv for future use
season_2018_df.to_csv('wr_season_2018_clean.csv', index=False)

In [42]:
# write the clean dataframes to csv for future use
season_2019_df.to_csv('wr_season_2019_clean.csv', index=False)

In [43]:
# write the clean dataframes to csv for future use
season_2020_df.to_csv('wr_season_2020_clean.csv', index=False)

In [44]:
# write the clean dataframes to csv for future use
season_2021_df.to_csv('wr_season_2021_clean.csv', index=False)

In [45]:
# check datatypes for creating sql table
season_2019_df.dtypes

Name                        object
Team                        object
Position                    object
Games                        int64
Targets                      int64
Receptions                   int64
Percentage                 float64
Yards                        int64
Touchdowns                   int64
Long                         int64
Yards_per_target           float64
Yards_per_reception        float64
Attempts                     int64
Rushing_yards                int64
Average_rushing_yards      float64
Rushing_touchdown            int64
Fumbles                      int64
Lost_yards                   int64
Fantasy_points_per_game    float64
Fantasy_points             float64
Year                        object
dtype: object

## Data Preprocessing

In [46]:
# drop unnecessary columns before fitting model
results = results.drop(columns=['Team', 'Position', 'Year', 'Games'])

In [47]:
# set name column as index
results = results.set_index('Name')

In [48]:
# create bins and labels for touchdown column
touchdown_bins = [-1, 3, 6, 9, 12, 15, 100]
bin_labels = [1,2,3,4,5,6]

In [49]:
# view new df containing converted data
results

,Targets,Receptions,Percentage,Yards,Touchdowns,Long,Yards_per_target,Yards_per_reception,Attempts,Rushing_yards,Average_rushing_yards,Rushing_touchdown,Fumbles,Lost_yards,Fantasy_points_per_game,Fantasy_points
Name,,,,,,,,,,,,,,,,
Tyreek Hill,137,87,63.5,1479,12,75,10.8,17.0,22,151,6.9,1,0,0,15.1,241.0
Antonio Brown,168,104,61.9,1297,15,78,7.7,12.5,0,0,0.0,0,0,0,14.6,219.7
Davante Adams,169,111,65.7,1386,13,57,8.2,12.5,0,0,0.0,0,0,0,14.6,218.6
DeAndre Hopkins,163,115,70.6,1572,11,49,9.6,13.7,1,-7,-7.0,0,2,2,13.7,218.5
Julio Jones,170,113,66.5,1677,8,58,9.9,14.8,2,12,6.0,0,2,2,13.3,212.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Malik Taylor,3,2,66.7,14,0,7,4.7,7.0,0,0,0.0,0,1,1,-0.1,-0.6
Racey McMath,6,2,33.3,8,0,9,1.3,4.0,0,0,0.0,0,1,1,-0.1,-1.2
Travis Benjamin,5,0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,1,1,-0.2,-2.0


In [50]:
results.shape

(990, 16)

In [52]:
# create the bins in new columbs for number of touchdowns
results['Touchdown_bins'] = pd.cut(results['Touchdowns'], bins = touchdown_bins, labels = bin_labels)


In [53]:
# y is the target and x is the features
# for this case we're training on touchdown performance first
y = results['Touchdown_bins']
X = results.drop(columns=['Touchdown_bins', 'Touchdowns'])

# create the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=.20)

In [54]:
# create a StandardScaler instance
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
X_test.shape

(198, 15)

## Model Creation

In [56]:
# define the model
nn = Sequential()

# first hidden layer
nn.add(Dense(units=70, activation = 'relu', input_dim = X_test.shape[1]))

# second hidden layer
nn.add(Dense(units=40, activation='relu'))

# third hidden layer
nn.add(Dense(units=20, activation='relu'))

# output layer
nn.add(Dense(units=1, activation='relu'))

# check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                1120      
                                                                 
 dense_1 (Dense)             (None, 40)                2840      
                                                                 
 dense_2 (Dense)             (None, 20)                820       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4801 (18.75 KB)
Trainable params: 4801 (18.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [57]:
# compile the model
# mae or mse
nn.compile(loss="mse", optimizer="adam", metrics=["accuracy"])

In [58]:
# train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=60)

Epoch 1/60
25/25 [==============================] - 0s 627us/step - loss: 0.5099 - accuracy: 0.5960
Epoch 2/60
25/25 [==============================] - 0s 467us/step - loss: 0.2377 - accuracy: 0.7551
Epoch 3/60
25/25 [==============================] - 0s 525us/step - loss: 0.1887 - accuracy: 0.7601
Epoch 4/60
25/25 [==============================] - 0s 562us/step - loss: 0.1627 - accuracy: 0.7601
Epoch 5/60
25/25 [==============================] - 0s 555us/step - loss: 0.1475 - accuracy: 0.7601
Epoch 6/60
25/25 [==============================] - 0s 561us/step - loss: 0.1310 - accuracy: 0.7601
Epoch 7/60
25/25 [==============================] - 0s 564us/step - loss: 0.1142 - accuracy: 0.7601
Epoch 8/60
25/25 [==============================] - 0s 560us/step - loss: 0.1018 - accuracy: 0.7601
Epoch 9/60
25/25 [==============================] - 0s 554us/step - loss: 0.0892 - accuracy: 0.7601
Epoch 10/60
25/25 [==============================] - 0s 550us/step - loss: 0.0763 - accuracy: 0.7601

In [59]:
# evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

7/7 - 0s - loss: 0.0659 - accuracy: 0.7475 - 67ms/epoch - 10ms/step
Loss: 0.06594718992710114, Accuracy: 0.747474730014801
